# Definitions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import regex as re
import glob
import errno    
import os
import distutils.dir_util
import datetime
import time
%matplotlib inline

one_hot_dict = { # Dictionary for turning a sequence into one hot encoding
    'A':[1,0,0,0],
    'C':[0,1,0,0],
    'G':[0,0,1,0],
    'T':[0,0,0,1],
    'a':[1,0,0,0],
    'c':[0,1,0,0],
    'g':[0,0,1,0],
    't':[0,0,0,1]
}

def one_hot_seq(seq):
    ''' Converts a sequence from ACTG into one hot format'''
    one_hot = []
    for bp in seq:
        one_hot += one_hot_dict[bp]
    return one_hot

def get_gpff_origin(gpff_file):
    '''extracts the primary sequence from a genbank genome'''
    with open(gpff_file, 'r') as file:
        seqs = []
        in_origin = False
        for line in file:
            # removes surrounding whitespace
            line = line.replace('\n','').strip()
            if line == 'ORIGIN':
                seq = ''
                in_origin = True

            if line == '//':
                seqs.append(seq)
                in_origin = False

            if in_origin:
                seq += ''.join(line.split(' ')[1:])
    
    return(seqs)

def read_seq_text(text_file):
    ''' Returns the information from a text file'''
    with open(text_file, 'r') as myfile:
        data = myfile.read().replace('\n', '')
    return(data)

def read_seq_fa(fa_file):
    ''' Returns the information from a FASTA file'''
    with open(fa_file, 'r') as myfile:
        data = myfile.read().replace('\n', '')
    on_target_sequence = data.split('gbk')[1]
    return(on_target_sequence)

def extract_genome(filename):
    ext = filename.split('.')[-1:][0]
    if ext == 'fa':
        # Read the insert from its fasta file
        genome = read_seq_fa(filename)
    elif ext == 'gb':
        # Read the insert from its fasta file
        genome = get_gpff_origin(filename)[0]
    else:
        print('Unknown filetype, please use fa or gb extension')
        return(False)
    return(genome)

def findPAMsites(genome):
    old_chars = "ACGTacgt" # Characters for forward sequence
    replace_chars = "TGCAtgca" # Characters for reverse sequence
    
    '''takes a genome, and finds all the potential sites for cas9 binding, saving index'''
    PAMs = []
    L = len(genome)
    # Find all sequences with a valid or alternate PAM site
    for PAM in re.finditer('([ATGC]{20})[ATCG][AG]G', genome, overlapped=True, flags=re.IGNORECASE):
        region = (PAM.span()[0], PAM.span()[1]-3)
        PAMs.append((PAM.group()[0:20], *region, PAM.group()[-3:].upper(), '+'))

    tab = str.maketrans(old_chars,replace_chars) # defines the translation map (A=T, C=G)
    rev_comp = genome.translate(tab)[::-1] # applies it, and flips around

    for PAM in re.finditer('([ATGC]{20})[ATCG][AG]G', rev_comp, overlapped=True, flags=re.IGNORECASE):
        region = (L-PAM.span()[1]+3, L-PAM.span()[0])
        PAMs.append((PAM.group()[0:20], *region, PAM.group()[-3:].upper(), '-'))

    return(pd.DataFrame(PAMs, columns=['gRNA', 'start', 'end', 'PAM', 'strand']))

def pair_distances(PAM_df, start_name, start_column, min_dist, max_dist):
    PAMs_sorted = PAM_df.sort_values(start_name)
    x = PAMs_sorted.values
    n = len(x)
    paired_gRNA = []

    for i in range(n): 
        for j in range(i+1,n):
            distance = abs(int(x[i][start_column]) - int(x[j][start_column]))
            if (min_dist < distance < max_dist) & (x[i][4] == x[j][4]) & (x[i][3][-2:] == 'GG') & (x[j][3][-2:] == 'GG'): # Check distance
                paired_gRNA.append([x[i], x[j], distance])
    return(paired_gRNA)

def off_targets_for_match(num_match, unique_seqs_in, genome):
    ''' Finds all the off-targets for a list of sequences with a seed match of num_match'''
    off_target_matches = []
    PAMs_off_target = []
    L = len(genome)
    
    # Find the off-target PAM sites
    PAMs_off_target_df = findPAMsites(genome)
    mycols = ['Seq', 'Hit_Start', 'Hit_End', 'PAM', 'Strand']
    PAMs_off_target_df.columns = mycols
    PAMs_off_target_df.loc[:, 'Seed'] = PAMs_off_target_df['Seq'].apply(lambda x: x.upper()[-num_match:])

    # Find the off-targets with num_match length seed for each sequence in unique_seqs_in
    for seq_on in unique_seqs_in:
        # Compare the first [num_match] characters of the sequence for an exact match then report as off-target
        seed_matching_rows = PAMs_off_target_df.loc[PAMs_off_target_df['Seed'] == seq_on[-num_match:]]
        seed_matching_rows.loc[:, 'Seq'] = seed_matching_rows['Seq'].apply(lambda x: x.upper())
        for index, seq_off_target in seed_matching_rows.iterrows():
            off_target_matches.append(np.append(seq_on, seq_off_target))
            
    return(off_target_matches)

def find_off_target_false_positives(root_out_path, genome_file, unique_seqs_in, dist_min, dist_max):

    now = datetime.datetime.now()
    date = str(now.date()).replace('-', '_')
    name_date_pairs = str(out_filename) + '_' + date + '_pairs/'
    
    # Set the genome file and on-target sequences
    genome_name = genome_file.split('/')[-1].split('.')[0]
    
    total_out_path = str(root_out_path) + str(genome_name) + '/'
    distutils.dir_util.mkpath(total_out_path)

    genome_off_target = get_gpff_origin(genome_file)[0]
    on_target_seqs = unique_seqs_in
    total_seed_on_target_seqs_false_pos = []
    mycols_off_targets = ['Seq_On_Target', 'Seq_Off_Target', 'Hit_Start', 'Hit_End', 'PAM', 'Strand', 'Seed']
    mycols_pairs = ['Seq_On_Target_1', 'Seq_Off_Target_1', 'Hit_Start_1', 'Hit_End_1', 'PAM_1', 'Strand_1', 'Seed_1',
                        'Seq_On_Target_2', 'Seq_Off_Target_2', 'Hit_Start_2', 'Hit_End_2', 'PAM_2', 'Strand_2', 'Seed_2']
    mycols_total_seed_false_pos = ['Seed', 'On_Target_Sequences']
    
    for num_match in [6, 7, 8]:
        # Set folder names based on num_match
        out_foldername = 'seq_offtargets' + '-' + str(num_match) + '_match'
        out_path = r'./' + name_date_pairs + genome_name + '/' + str(out_foldername) + '/'
        distutils.dir_util.mkpath(out_path)

        
        # Find off_targets for all the sequences in the on-target pairs with seed length num_match
        off_targets = off_targets_for_match(num_match, on_target_seqs, genome_off_target)
        
        # Place off-targets into a dataframe
        off_targets_df = pd.DataFrame(pd.np.empty((0, len(mycols_off_targets))))
        if len(off_targets) != 0:
            off_targets_df = pd.DataFrame(off_targets)
        off_targets_df.columns = mycols_off_targets

        
        # Sort dataframe for pairing
        off_targets_df_sorted = off_targets_df.sort_values(['Seq_On_Target','Hit_Start'])
        off_targets_df_sorted.to_csv(str(out_path) + 'off_targets_' + str(genome_name) + '_' + str(num_match) + '.csv')
        false_pairs_total = []

        # Find all pairs within the off-targets of each on-target sequence
        paired_gRNA = []
        for curr_seq in unique_seqs_in:
            x = off_targets_df.sort_values(['Seq_On_Target','Hit_Start']).loc[off_targets_df['Seq_On_Target'] == curr_seq].values
            n = len(x)
            for i in range(n): 
                for j in range(i+1,n):
                    distance = abs(int(x[i][2]) - int(x[j][2]))
                    if (dist_min < distance < dist_max): # Check distances for specifics
                        paired_gRNA.append(np.append(x[i], x[j]))

                        
        # If there exist no pairs for num_match = 6, return the blank results out and complete off targeting
        if len(paired_gRNA) <= 0:
            # Set the other off-target seed lengths that need to be computed and add the empty entries
            if num_match == 6:
                off_target_seeds = [8, 10]
                total_seed_on_target_seqs_false_pos.append([6, []])
                total_seed_on_target_seqs_false_pos.append([7, []])
                total_seed_on_target_seqs_false_pos.append([8, []])
            elif num_match == 7:
                off_target_seeds = [8, 10]
                total_seed_on_target_seqs_false_pos.append([7, []])
                total_seed_on_target_seqs_false_pos.append([8, []])
            else:
                off_target_seeds = [10]
                total_seed_on_target_seqs_false_pos.append([8, []])
                
            # Find off-targets for all the sequences in the on-target pairs with seed length num_match
            for internal_num_match in off_target_seeds:
                # Set folder names based on internal_num_match
                out_foldername = 'seq_offtargets' + '-' + str(internal_num_match) + '_match'
                out_path = r'./'  + name_date_pairs + genome_name + '/' + str(out_foldername) + '/'
                distutils.dir_util.mkpath(out_path)

                # Get off targets for internal_num_match then sort in DataFrame for printing to csv
                off_targets = off_targets_for_match(internal_num_match, on_target_seqs, genome_off_target)
                if len(off_targets) <= 0:
                    off_targets_df = pd.DataFrame([['NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']])
                else:
                    off_targets_df = pd.DataFrame(off_targets)
                off_targets_df.columns = mycols_off_targets
                off_targets_df_sorted = off_targets_df.sort_values(['Seq_On_Target','Hit_Start'])
                off_targets_df_sorted.to_csv(str(out_path) + 'off_targets_' + str(genome_name) + '_' + str(internal_num_match) + '.csv')

            total_seed_on_target_seqs_false_pos_df = pd.DataFrame(total_seed_on_target_seqs_false_pos)
            mycols_total_seed_false_pos = ['Seed', 'On_Target_Sequences']
            total_seed_on_target_seqs_false_pos_df.columns = mycols_total_seed_false_pos
            total_seed_on_target_seqs_false_pos_df.to_csv(str(total_out_path) + str(genome_name) + '_seed_on_target_false_pos.csv')
            return(total_seed_on_target_seqs_false_pos_df)

        # Set up a dataframe of the results
        off_target_pairs_df = pd.DataFrame(pd.np.empty((0, len(mycols_pairs))))  
        off_target_pairs_df = pd.DataFrame(paired_gRNA)
        off_target_pairs_df.columns = mycols_pairs

        # Check that the on_target sequences are lined up properly and do not contain more values than the unique on-target sequences, otherwise issue a warning
        assert (set(off_target_pairs_df['Seq_On_Target_1']) == set(off_target_pairs_df['Seq_On_Target_2'])) == True
        assert len(set(off_target_pairs_df['Seq_On_Target_1'].values)) <= len(unique_seqs_in)

        # Store the on-target sequences with false positive pairs
        on_target_seqs_false_pos = set(off_target_pairs_df['Seq_On_Target_1'].values)
        total_seed_on_target_seqs_false_pos.append(np.append(num_match, on_target_seqs_false_pos))

    # Set folder names for the 10 match
    out_foldername = 'seq_offtargets-10_match'
    out_path = r'./' + name_date_pairs + genome_name + '/' + str(out_foldername) + '/'
    distutils.dir_util.mkpath(out_path)

    # Get off targets for seed length of 10 then sort in DataFrame for printing to csv
    off_targets = off_targets_for_match(10, on_target_seqs, genome_off_target)
    if len(off_targets) <= 0:
        off_targets_df = pd.DataFrame([['NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']])
    else:
        off_targets_df = pd.DataFrame(off_targets)
    off_targets_df.columns = mycols_off_targets
    off_targets_df_sorted = off_targets_df.sort_values(['Seq_On_Target','Hit_Start'])
    off_targets_df_sorted.to_csv(str(out_path) + 'off_targets_' + str(genome_name) + '_10.csv')
    
    # Report total findings
    total_seed_on_target_seqs_false_pos_df = pd.DataFrame(total_seed_on_target_seqs_false_pos)
    total_seed_on_target_seqs_false_pos_df.columns = mycols_total_seed_false_pos
    total_seed_on_target_seqs_false_pos_df.to_csv(str(total_out_path) + str(genome_name) + '_seed_on_target_false_pos.csv')
    
    return(total_seed_on_target_seqs_false_pos_df)

# Converts the seed length for off-target pairs to weights for scoring
def convert_seed_weight(row, seed_weights):
    if row['FP_Seed'] == 6:
        return(seed_weights[0])
    elif row['FP_Seed'] == 7:
        return(seed_weights[1])
    elif row['FP_Seed'] == 8:
        return(seed_weights[2])
    else:
        return(seed_weights[3])

# Ensures the score is within allowed values for guides
def guide_scoring(row):
    score = row['sgRNA_Score']
    if (score >= 1):
        return(1.0)
    elif (score <= 0):
        return(2.0)
    else:
        return(round(score, 3))

# Find the on-target gRNA pairs for a given target genome

In [ ]:
# Specify distances for on-target pairs
min_dist_on_target = 23
max_dist_on_target = 84

# Specify distances for off-target pairs to avoid
off_target_pair_min = 23
off_target_pair_max = 200

# Scoring weights
seed_weights = [2, 4, 6, 8]
w_10 = 250 # Each match of seed length 10 is weighted as w_10
w_8 = 50 # Each match of seed length 8 is weighted as w_8
w_6 = 1 # Each match of seed length 6 is weighted as w_6

# Change to the folder where you have target sequences
target_sequence_folder = './individual_target_sequences'

# Select folders where off-target genomes are contained
genome_dir = '../../Sequences/genomes_for_sensor_optimization/genomes'

# Set name of the output folder
out_foldername = 'paired_gRNA_on_target'


# ================================================================
for i, filename in enumerate(glob.glob(target_sequence_folder + '/*.fa')):  
    # Set file and folder names
    now = datetime.datetime.now()
    out_filename = filename.split('/')[-1].split('.')[0].replace('\s','_')
    date = str(now.date()).replace('-', '_')
    name_date_pairs = str(out_filename) + '_' + date + '_pairs/'
    out_path = './' + name_date_pairs + str(out_foldername) + '/'
    distutils.dir_util.mkpath(out_path)

    # Extract genome from file
    print(filename)
    genome_target = extract_genome(filename)

    # Find the PAM sites within the insert
    PAMs = findPAMsites(genome_target)
    
    # Find all the paired guides between minimum and maximum distances with 'NGG' and on the same strand
    paired_gRNA = pair_distances(PAMs, 'start', 1, min_dist_on_target, max_dist_on_target)
    
    if len(paired_gRNA) <= 0:
        print(str(filename) + ' has no on-target pairs with the current criteria!')
        continue
    
    # Place table output into a csv, sorted by Location distance and ordered for output    
    out_namecsv = out_path + out_filename + '-results' + '.csv'
    paired_df = pd.DataFrame(paired_gRNA)
    paired_df.columns = ['gRNA_1', 'gRNA_2',  'distance']
    paired_df = paired_df.sort_values('distance', ascending = True)
    paired_df.to_csv(out_namecsv, index=False)
    print(out_namecsv + ' complete!')
    print('#On-target pairs = ' + str(len(paired_gRNA)))
    paired_df.hist()
    plt.savefig('./' + name_date_pairs + '/on_target_distance_hist.png')
        
    
    # Separate unique sequences for finding off-targets
    seqs_in = [] # To be used for finding off-targets
    for pair in paired_df['gRNA_1']:
        seqs_in.append(pair[0])
    for pair in paired_df['gRNA_2']:
        seqs_in.append(pair[0])
    unique_seqs_in = set(seqs_in)
    print('#Unique Sequences = ' + str(len(unique_seqs_in)))
    
    start = time.time() # Timing for the functions
    root_out_path = './' + name_date_pairs
    input_unique_seqs = unique_seqs_in
    
    # Find the off-target false positives for all the genomes in the genome_dir directory
    for i, genome_genbank_file in enumerate(glob.glob(genome_dir + '/*.gb')):
        find_off_target_false_positives(root_out_path, genome_genbank_file, input_unique_seqs, off_target_pair_min, off_target_pair_max)
        print(genome_genbank_file)

    # Timing for large genome runtime
    end_genome = time.time()
    print(end_genome - start)
    
    # ==============================================================
    # Set the date manually
    # ==============================================================

    set_date = date # Modify if wanting to combine targets from another date
    name_set_date_pairs = str(out_filename) + '_' + set_date + '_pairs/'

    # ==============================================================
    # Ordering and printing results
    # ==============================================================

    # Initialize arrays
    seed_list = []
    seed_dict = []
    total_false_pos_6_match = []
    total_false_pos_7_match = []
    total_false_pos_8_match = []
    non_viable_6_seqs = []
    non_viable_7_seqs = []
    non_viable_8_seqs = []
    
    # Collect all the false positive pair sequences for several seed lengths
    for i, false_pos_file in enumerate(glob.glob('./' + name_set_date_pairs + '*/*pos.csv')):  
        curr_df = pd.DataFrame.from_csv(false_pos_file)
        total_false_pos_6_match.append(curr_df['On_Target_Sequences'][0])
        total_false_pos_7_match.append(curr_df['On_Target_Sequences'][1])
        total_false_pos_8_match.append(curr_df['On_Target_Sequences'][2])
        
    # Find all the non-viable sequences
    for on_target_seq in unique_seqs_in:
        for seqs in total_false_pos_6_match:
            if on_target_seq in seqs:
                non_viable_6_seqs.append(on_target_seq)

        for seqs in total_false_pos_7_match:
            if on_target_seq in seqs:
                non_viable_7_seqs.append(on_target_seq)

        for seqs in total_false_pos_8_match:
            if on_target_seq in seqs:
                non_viable_8_seqs.append(on_target_seq)

    # Create lists of the sequences with no false positive pairs in each seed length
    viable_seqs_6_match = unique_seqs_in.difference(non_viable_6_seqs)
    viable_seqs_7_match = unique_seqs_in.difference(non_viable_7_seqs)
    viable_seqs_8_match = unique_seqs_in.difference(non_viable_8_seqs)

    # Initialize the dataframe
    total_paired_rows = []
    for i, row_i in paired_df.iterrows():
        curr_row = np.append(np.append(row_i[0], row_i[1]), row_i[2])
        total_paired_rows.append(curr_row)
    total_paired_df = pd.DataFrame(total_paired_rows)

    # Define the dataframe columns
    mycols_total_paired =  ['Seq_On_Target_1', 'Hit_Start_1', 'Hit_End_1', 'PAM_1', 'Strand_1', 
                            'Seq_On_Target_2', 'Hit_Start_2', 'Hit_End_2', 'PAM_2', 'Strand_2', 'Distance']
    mycols_total_combined =  ['Seq_On_Target_1', 'Index_1', 'PAM_Pos_1', 
                              'Seq_On_Target_2', 'Index_2', 'PAM_Pos_2', 
                              'FP_Seed', 'Distance']

    # Set the columns for the dataframe
    total_paired_df.columns = mycols_total_paired

    # Separate values by length of smallest false positive
    total_paired_df.loc[(total_paired_df['Seq_On_Target_1'].isin(viable_seqs_8_match)) & (total_paired_df['Seq_On_Target_2'].isin(viable_seqs_8_match)), 'FP_Seed'] = 8
    total_paired_df.loc[(total_paired_df['Seq_On_Target_1'].isin(viable_seqs_7_match)) & (total_paired_df['Seq_On_Target_2'].isin(viable_seqs_7_match)), 'FP_Seed'] = 7
    total_paired_df.loc[(total_paired_df['Seq_On_Target_1'].isin(viable_seqs_6_match)) & (total_paired_df['Seq_On_Target_2'].isin(viable_seqs_6_match)), 'FP_Seed'] = 6
    total_paired_df.loc[:,'FP_Seed'] = total_paired_df['FP_Seed'].fillna('9+')

    # Define combined columns for simple viewing
    total_combined_df = total_paired_df
    total_combined_df.loc[:, 'Index_1'] = total_paired_df['Hit_Start_1'].map(str) + '-' + total_paired_df['Hit_End_1'].map(str)
    total_combined_df.loc[:, 'Index_2'] = total_paired_df['Hit_Start_2'].map(str) + '-' + total_paired_df['Hit_End_2'].map(str)
    total_combined_df.loc[:, 'PAM_Pos_1'] = total_paired_df['PAM_1'].map(str) + '/' + total_paired_df['Strand_1'].map(str)
    total_combined_df.loc[:, 'PAM_Pos_2'] = total_paired_df['PAM_2'].map(str) + '/' + total_paired_df['Strand_2'].map(str)

    # Select just the combined columns
    total_expanded_df = total_combined_df.loc[:, ]
    total_combined_df = total_combined_df.loc[:, mycols_total_combined]

    # Search for the counts for singular off-targets for each of the sequences
    for curr_num_match in [6, 8, 10]:
        off_targets_df = pd.DataFrame.from_csv('./' + name_set_date_pairs + 'genome1/seq_offtargets-' + str(curr_num_match) + '_match/off_targets_genome1_' + str(curr_num_match) + '.csv')
        for curr_seq in unique_seqs_in:
            seed_list.append([(curr_seq, curr_num_match), off_targets_df.loc[off_targets_df['Seq_On_Target'] == curr_seq].shape[0]])

    # Create a dictionary of the seed counts for easy access
    seed_dict = dict(seed_list)

    # Calculate the combined off-target values
    for curr_num_match in [6, 8, 10]:    
        # Define expanded values
        column_name = 'Seq_1_Off_' + str(curr_num_match)
        total_expanded_df.loc[:, column_name] = total_paired_df.apply(lambda x: seed_dict[(x['Seq_On_Target_1'], curr_num_match)], axis = 1)
        column_name = 'Seq_2_Off_' + str(curr_num_match)
        total_expanded_df.loc[:, column_name] = total_paired_df.apply(lambda x: seed_dict[(x['Seq_On_Target_2'], curr_num_match)], axis = 1)
        # Define combined values
        column_name = 'Off_' + str(curr_num_match)
        total_combined_df.loc[:, column_name] = total_paired_df.apply(lambda x: int(seed_dict[(x['Seq_On_Target_1'], curr_num_match)]) + int(seed_dict[(x['Seq_On_Target_2'], curr_num_match)]), axis = 1)

    # Remove combined columns for expanded
    del total_expanded_df['Index_1'], total_expanded_df['Index_2'], total_expanded_df['PAM_Pos_1'], total_expanded_df['PAM_Pos_2']
    total_expanded_df.sort_values(['FP_Seed', 'Distance']).to_csv('./' + name_set_date_pairs + 'Ranked_Guide_Pairs_' + out_filename + '_expanded.csv')
    


    # Scoring for the guides
    total_combined_df.loc[:, 'FP_Score'] = total_combined_df.apply(lambda row: convert_seed_weight(row, seed_weights), axis = 1)
    sum_off_targets = (total_combined_df['Off_10'] + total_combined_df['Off_8'] + total_combined_df['Off_6'])
    avg_sum_w = (w_10 + w_8 + w_6)/3
    total_combined_df.loc[:, 'sgRNA_Score'] = (total_combined_df['FP_Score']/(sum_off_targets*avg_sum_w))*((w_10 * total_combined_df['Off_10']) + (w_8 * total_combined_df['Off_8']) + (w_6 * total_combined_df['Off_6']))
    total_combined_df.loc[:, 'sgRNA_Score'] = total_combined_df.apply(lambda row: guide_scoring(row), axis = 1)
    del total_combined_df['FP_Score']
    
    # Send the dataframes to csv
    total_combined_df.sort_values('sgRNA_Score').to_csv('./' + name_set_date_pairs + 'Ranged_Guide_Pairs_' + out_filename + '_ranked.csv')
    total_combined_df.sort_values(['FP_Seed', 'Distance']).to_csv('./' + name_set_date_pairs + 'Ranked_Guide_Pairs_' + out_filename + '_combined.csv')